In [24]:
# load package
import numpy as np
import pandas as pd
from collections import Counter
import lightgbm as lgb
#from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import pickle

# set path
import os
default_path = "/Users/francislin/JrML/03_04_What's Cooking/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [25]:
with open('./data/all_data_emb_k32.pickle','rb') as f:
    df= pickle.load(f)

In [26]:
# 區分train test
dataTrain = df[pd.notnull(df['cuisine'])]
dataTest = df[~pd.notnull(df['cuisine'])]
dataTrain.columns
del(df)

Index(['id', 'cuisine', 'greek_0', 'greek_1', 'greek_2', 'greek_3', 'greek_4',
       'greek_5', 'greek_6', 'greek_7',
       ...
       'russian_22', 'russian_23', 'russian_24', 'russian_25', 'russian_26',
       'russian_27', 'russian_28', 'russian_29', 'russian_30', 'russian_31'],
      dtype='object', length=642)

In [27]:
#dataTrain = dataTrain
#去掉y
dataTest = dataTest.drop(['id','cuisine'],axis=1)
#去掉key
X_feature = dataTrain.drop(['id','cuisine'],axis=1)
y_t =dataTrain['cuisine']

In [28]:
X_feature.head()

,greek_0,greek_1,greek_2,greek_3,greek_4,greek_5,greek_6,greek_7,greek_8,greek_9,...,russian_22,russian_23,russian_24,russian_25,russian_26,russian_27,russian_28,russian_29,russian_30,russian_31
0,1.356402e-07,1.658690e-07,7.074824e-08,1.529251e-07,5.191751e-08,1.183570e-07,1.032993e-07,7.768111e-08,1.485513e-07,1.225367e-07,...,-2.928500e-08,-3.011975e-09,3.551468e-08,-9.313183e-09,1.027598e-08,-3.201952e-09,2.477666e-08,2.400870e-08,1.004919e-08,4.257494e-08
1,-3.599593e-08,-3.449493e-08,-4.894840e-08,-3.075692e-08,-3.375822e-08,-3.766603e-08,-4.000164e-08,-4.445520e-08,-2.624439e-08,-4.027262e-08,...,1.859867e-09,8.810141e-08,1.088635e-07,4.218034e-08,5.461292e-08,4.633686e-08,1.138715e-07,8.886790e-08,9.745802e-08,1.208372e-07
2,1.103896e-05,1.050039e-05,4.419902e-06,5.689063e-06,7.637005e-06,4.067915e-06,1.024807e-05,4.095847e-06,1.612509e-06,5.657254e-06,...,3.615708e-06,8.815463e-06,6.749708e-06,1.095223e-05,7.768596e-06,9.094721e-06,2.476099e-06,8.813379e-06,2.013316e-06,9.461340e-06
3,-2.019742e-08,-1.815091e-08,-1.837140e-08,-1.285076e-08,-1.221861e-08,-2.057284e-08,-1.781205e-08,-2.021390e-08,-1.176155e-08,-1.887218e-08,...,1.787777e-08,1.438721e-07,1.233514e-07,7.295086e-08,6.388265e-08,7.278023e-08,1.536631e-07,1.142229e-07,1.402663e-07,1.434954e-07
4,3.954460e-05,1.443133e-05,1.167779e-05,3.141066e-05,2.526310e-05,3.226248e-05,1.061126e-05,3.342805e-05,4.641153e-05,1.535157e-05,...,1.498320e-05,2.908074e-05,1.741402e-05,5.240723e-05,5.091304e-05,4.274395e-05,2.789780e-05,2.428468e-05,5.132026e-05,4.562938e-05


In [29]:
#把上面dataTrain.columns的結果貼到dataTrain，再去掉'Survived'
from sklearn.model_selection import train_test_split
#拿掉feature_importances低的因子
X_train, X_test, y_train, y_test = train_test_split(X_feature, y_t, test_size=0.3, random_state=13)

In [30]:
# Function for Measure Performance
from  sklearn  import  metrics
def measure_performance(X,y,clf, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True, show_roc_auc = True):
    y_pred = clf.predict(X)
    y_predprob = clf.predict_proba(X)[:,1]
    if show_accuracy:
        print ("Accuracy:{0:.3f}".format(metrics.accuracy_score(y,y_pred))),"\n"

    if show_classification_report:
        print("Classification report")
        print(metrics.classification_report(y,y_pred)),"\n"
        
    if show_confusion_matrix:
        print("Confusion matrix")
        print(metrics.confusion_matrix(y,y_pred)),"\n"  
        
    if show_roc_auc:
        print("ROC AUC Score")
        print(metrics.roc_auc_score(y,y_predprob)),"\n"

In [31]:
X_train_array = X_train.values
X_test_array = X_test.values

In [32]:
%%time
import lightgbm as lgb
lgb = lgb.LGBMClassifier(object='multiclass', num_class=20, max_depth=8,learning_rate=0.25)
lgb_fit=lgb.fit(X_train,y_train)

CPU times: user 7min 56s, sys: 9.22 s, total: 8min 5s
Wall time: 2min 16s


In [33]:
y_pred= lgb_fit.predict(X_test)

In [34]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.52525253, 0.26911315, 0.55319149, 0.71977401, 0.39583333,
       0.48700063, 0.59931507, 0.78947368, 0.41423948, 0.7531883 ,
       0.64028777, 0.64900662, 0.61339093, 0.82693774, 0.54807692,
       0.28019324, 0.61080711, 0.28644501, 0.62406816, 0.42307692])

0.5504336050637574

0.6680633537249644

In [88]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[  72,    1,    2,    1,    3,    4,    0,    0,    0,   20,    0,
           1,    0,   13,    0,    0,   11,    1,    1,    1],
       [   0,   68,    2,    1,    0,   25,    1,    4,   17,   31,    3,
           0,    0,    3,    0,    4,   60,    0,    0,    0],
       [   0,    2,  309,    1,    3,   17,    1,    1,    0,   28,    1,
           0,    0,   20,    1,    0,   70,    2,    0,    0],
       [   1,    0,    2,  672,    6,    9,    2,    3,    0,   11,    0,
          22,   21,   10,    1,    0,   32,    0,   22,   14],
       [  10,    0,    0,   34,  130,    3,    0,    2,    0,   13,    0,
           3,    1,    6,    0,    0,   29,    1,    9,    8],
       [   1,    7,    7,    2,    3,  437,    7,    2,   11,  176,    3,
           1,    0,    6,    3,    9,   68,   11,    1,    1],
       [   0,    1,    0,    0,    0,    5,  240,    7,    0,   71,    0,
           0,    1,    6,    4,    0,    6,    5,    0,    0],
       [   1,    1,    0,    4,    0,    

In [40]:
# Grid Search
import lightgbm as lgb
print('Start training...')
estimator = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0) 

param_grid = {
    'learning_rate': [0.11], 
    'num_leaves': [64],
    'feature_fraction': [0.8],
    'bagging_fraction': [0.9],
    'max_depth':[9],
    #'max_bin': [6],
    'bagging_freq':[4],
    'lambda_l1': [20] , #[44,133] then 44 [44,88] then 44 np.arange(38,44,1) np.arange(16,20,1)
    'lambda_l2': [7] ,#np.arange(11,15,1) then 13
    #'min_split_gain': [0.04], 
    'min_child_weight': [2] 
}

LGBM_grid = GridSearchCV(estimator, param_grid,scoring='accuracy')#scoring='f1_macro'
LGBM_grid.fit(X_train, y_train)
print('Best parameters found by grid search are:', LGBM_grid.best_params_)

# Final Model
evals_result = {} 
print('Start predicting...')
LGBM_grid_final = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0,
                                     learning_rate = LGBM_grid.best_params_['learning_rate'],
                              num_leaves = LGBM_grid.best_params_['num_leaves'],
                                feature_fraction = LGBM_grid.best_params_['feature_fraction'], 
                             bagging_fraction = LGBM_grid.best_params_['bagging_fraction'],
                             max_depth = LGBM_grid.best_params_['max_depth'],
                              #max_bin = LGBM_grid.best_params_['max_bin'],
                                bagging_freq = LGBM_grid.best_params_['bagging_freq'],
                              lambda_l1 = LGBM_grid.best_params_['lambda_l1'],
                              lambda_l2 = LGBM_grid.best_params_['lambda_l2'],
                            #min_split_gain = LGBM_grid.best_params_['min_split_gain'],
                            min_child_weight = LGBM_grid.best_params_['min_child_weight']
                                    )
LGBM_grid_final_fit = LGBM_grid_final.fit(X_train, y_train)
print("Predicting is over")

Start training...


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_class=20, num_leaves=31, object='multiclass',
        objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.11], 'num_leaves': [64], 'feature_fraction': [0.8], 'bagging_fraction': [0.9], 'max_depth': [9], 'bagging_freq': [4], 'lambda_l1': [20], 'lambda_l2': [7], 'min_child_weight': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

Best parameters found by grid search are: {'bagging_fraction': 0.9, 'bagging_freq': 4, 'feature_fraction': 0.8, 'lambda_l1': 20, 'lambda_l2': 7, 'learning_rate': 0.11, 'max_depth': 9, 'min_child_weight': 2, 'num_leaves': 64}
Start predicting...
Predicting is over


In [41]:
#對test算performance
LGBM_grid_measure = measure_performance(X = X_test, y = y_test, clf = LGBM_grid_final, show_classification_report=True, show_confusion_matrix=True)

Accuracy:0.639
Classification report
              precision    recall  f1-score   support

   brazilian       0.78      0.33      0.46       131
     british       0.39      0.12      0.18       219
cajun_creole       0.66      0.43      0.52       456
     chinese       0.65      0.74      0.69       828
    filipino       0.57      0.26      0.35       249
      french       0.45      0.48      0.47       756
       greek       0.70      0.45      0.55       346
      indian       0.74      0.78      0.76       925
       irish       0.49      0.27      0.35       185
     italian       0.64      0.82      0.72      2420
    jamaican       0.68      0.48      0.56       159
    japanese       0.68      0.56      0.62       420
      korean       0.68      0.47      0.56       245
     mexican       0.77      0.82      0.80      1934
    moroccan       0.56      0.38      0.46       242
     russian       0.55      0.15      0.24       145
 southern_us       0.52      0.71      0.60 

ValueError: multiclass format is not supported

In [45]:
#拿全部的dataTrain來建
LGBM_grid_final_fit_all = LGBM_grid_final.fit(X_feature, y_t)

In [50]:
LGBM_grid_final_fit_res =LGBM_grid_final_fit_all.predict(dataTest)

In [52]:
LGBM_grid_final_fit_res

array(['southern_us', 'southern_us', 'italian', ..., 'italian',
       'southern_us', 'mexican'], dtype=object)

In [53]:
submit=pd.read_csv('./data/sample_submission.csv')

In [54]:
submit['cuisine'] = LGBM_grid_final_fit_res

In [55]:
%pwd
submit.to_csv('submit_to_kaggle.csv', index= False)

"/Users/francislin/JrML/03_04_What's Cooking"

In [56]:
submit.cuisine[0]

'southern_us'

In [21]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=500,max_features=10)
rf_clf.fit(X_train, y_train)

CPU times: user 5min 15s, sys: 773 ms, total: 5min 16s
Wall time: 5min 16s


In [22]:
y_pred= rf_clf.predict(X_test)

In [23]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.36046512, 0.16551724, 0.17594255, 0.24508887, 0.1443299 ,
       0.24224806, 0.09045226, 0.32727273, 0.3032491 , 0.3918108 ,
       0.36966825, 0.41401274, 0.27160494, 0.32260692, 0.31152648,
       0.15819209, 0.3452381 , 0.18131868, 0.1858864 , 0.24316109])

0.2624796156636354

0.3333612670744993

In [30]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
GBDT = GradientBoostingClassifier(n_estimators=100,max_depth=5,learning_rate=0.1)#深度從3變5沒有比較好＠＠
GBDT.fit(X_train,y_train)

CPU times: user 20min 2s, sys: 5.13 s, total: 20min 7s
Wall time: 20min 13s


In [31]:
y_pred= GBDT.predict(X_test)

In [32]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.29347826, 0.1980198 , 0.63980464, 0.69152542, 0.43501326,
       0.44920525, 0.52406417, 0.76623377, 0.25342466, 0.75115634,
       0.52964427, 0.50886076, 0.57471264, 0.81545702, 0.56265985,
       0.20512821, 0.60767947, 0.35323383, 0.65959703, 0.45346062])

0.513617963345428

0.6549903628592978

In [35]:
%%time
from xgboost import XGBClassifier
Xgb = XGBClassifier()
Xgb.fit(X_train,y_train)

CPU times: user 31min 37s, sys: 5.38 s, total: 31min 43s
Wall time: 31min 57s


In [36]:
y_pred= Xgb.predict(X_test)

In [37]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.39772727, 0.13127413, 0.50909091, 0.68232386, 0.24464832,
       0.43806858, 0.52551985, 0.74904319, 0.33333333, 0.65324243,
       0.54474708, 0.60344828, 0.55684455, 0.78872524, 0.44077135,
       0.22857143, 0.54809315, 0.20178042, 0.59325843, 0.35772358])

0.4764117682303267

0.6113299254169111